## Parallelization

##### Load the environment Variables

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
tavily_api_key = os.getenv('TAVILY_API_KEY')
model_id = os.getenv('MODEL_ID')
aws_region = os.getenv('AWS_REGION')
bedrock_kb_id = os.getenv('BEDROCK_KB_ID')

##### Model

In [ ]:
## Model - Agent Brain
from langchain_aws import ChatBedrock
llm = ChatBedrock(model=model_id)

##### State - The schema of the State will be the input schema to all Nodes and Edges in the graph

In [ ]:
## Graph State
from typing import TypedDict

class State(TypedDict):
    query: str
    web_search: str
    kb_search: str
    final_answer: str

##### Tools [Tool 1 - Web Search Tool]

In [ ]:
import os
from langchain_community.tools.tavily_search import TavilySearchResults

def search_web(state: State):
    print("SEARCHING WEB")
    search_tool = TavilySearchResults(max_results=2)
    web_results = search_tool.invoke(state["query"])
    return {"web_search": web_results}

##### Tools [Tool 2 - Bedrock Knowledge Base]

In [ ]:
import boto3
def query_knowledge_base(state: State):
    """Query the knowledge base for information related to Agents and Agentic workflow
    
    Args:
        query: The query string to search for
    """
    bedrock_agent = boto3.client('bedrock-agent-runtime', region_name = 'us-east-1')
    print("QUERYING KNOWLEDGE BASE")

    response = bedrock_agent.retrieve_and_generate(
        input={
            "text": state["query"]  # Your query text goes here
        },
        retrieveAndGenerateConfiguration={
            "type": "KNOWLEDGE_BASE",
            "knowledgeBaseConfiguration": {
                "knowledgeBaseId": bedrock_kb_id,
                "modelArn": model_id,
                "retrievalConfiguration": {
                    "vectorSearchConfiguration": {
                        "numberOfResults": 5
                    }
                }
            }
        }
    )

    kb_results = response['output']['text']
    return {"kb_search" : kb_results}

##### Toos [Tool 3 - Aggregator tool]

In [ ]:
def aggregator(state: State):
    print("AGGREGATING RESPONSE")
    prompt = f""" Your job is to summarize from the context provided to you.  the context includes information from web search: {state["web_search"]} and information from database: {state["kb_search"]}"""
    final_answer = llm.invoke(prompt)           
    return {"final_answer": final_answer.content}                                    

##### Build the Graph

In [ ]:
from langgraph.graph import StateGraph, START, END

parallel_builder = StateGraph(State)

# Add nodes
parallel_builder.add_node("call_websearch", search_web)
parallel_builder.add_node("call_kb", query_knowledge_base)
parallel_builder.add_node("aggregator", aggregator)

parallel_builder.add_edge(START, "call_websearch")
parallel_builder.add_edge(START, "call_kb")

parallel_builder.add_edge("call_websearch", "aggregator")
parallel_builder.add_edge("call_kb", "aggregator")

parallel_builder.add_edge("aggregator", END)


##### Compile the Graph

In [ ]:
agent = parallel_builder.compile()

In [ ]:
import requests
from IPython.display import Image, display

requests.adapters.DEFAULT_TIMEOUT = 30  # Increase from default 10 seconds
display(Image(agent.get_graph().draw_mermaid_png()))

##### Run the agent

In [ ]:
state = agent.invoke({"query": "what are the ai agent types"})
print(state["final_answer"])